In [3]:
import json

with open('GSPC-Yahoo-Finance.json') as f:
    data = json.load(f)

In [4]:
import pandas as pd

df = pd.DataFrame()

df['timestamp'] = data['chart']['result'][0]['timestamp']

quotes = data['chart']['result'][0]['indicators']['quote'][0]

for quote in quotes.keys():
    df[quote] = quotes[quote]

df['adjclose'] = data['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']



In [5]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [6]:
df

,timestamp,volume,open,close,high,low,adjclose
0,1927-12-30 14:30:00,0,17.660000,17.660000,17.660000,17.660000,17.660000
1,1928-01-03 14:30:00,0,17.760000,17.760000,17.760000,17.760000,17.760000
2,1928-01-04 14:30:00,0,17.719999,17.719999,17.719999,17.719999,17.719999
3,1928-01-05 14:30:00,0,17.549999,17.549999,17.549999,17.549999,17.549999
4,1928-01-06 14:30:00,0,17.660000,17.660000,17.660000,17.660000,17.660000
...,...,...,...,...,...,...,...
23720,2022-06-06 13:30:00,3852050000,4134.720215,4121.430176,4168.779785,4109.180176,4121.430176
23721,2022-06-07 13:30:00,3476470000,4096.470215,4160.680176,4164.859863,4080.189941,4160.680176
23722,2022-06-08 13:30:00,3090180000,4147.120117,4115.770020,4160.140137,4107.200195,4115.770020
23723,2022-06-09 13:30:00,3192330000,4101.649902,4017.820068,4119.100098,4017.169922,4017.820068


In [7]:
# Calculate bear markets

inBearMkt = False
lastPeak = None
bearMarketThreshold = 0.2 # Decline since peak when the market is officially bearish

bearMarkets = []

for i, day in df.iterrows():
    if lastPeak == None:
        lastPeak = day.to_dict()

    if not inBearMkt:
        if day['close'] > lastPeak['close']:
            lastPeak = day.to_dict()
        if day['close'] <= lastPeak['close'] * (1-bearMarketThreshold):
            inBearMkt = True
            bearMarkets.append({'start': lastPeak['timestamp'], 'trigger': day['timestamp']})
    
    if inBearMkt:
        if day['close'] >= lastPeak['close']:
            bearMarkets[-1]['end'] = day['timestamp']
            inBearMkt = False


In [8]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x = df.timestamp, y = df.close))
fig.show()

In [9]:
from datetime import date

bearData = []

for bear in bearMarkets:
    mask = (df['timestamp'] >= bear['start']) & (df['timestamp'] <= bear['end'])
    bearData.append(df.loc[mask])

In [10]:
for data in bearData:
    data['days'] = range(len(data))
    pos = data.columns.get_loc('close')
    data['change'] =  (data.iloc[1:, pos] / data.iat[0, pos]) -1

C:\Users\ash\AppData\Local\Temp/ipykernel_15944/2008384127.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ash\AppData\Local\Temp/ipykernel_15944/2008384127.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
fig = go.Figure(data=go.Scatter(x = df.timestamp, y = df.close))
for i, bear in enumerate(bearMarkets):
	# Add a shape whose x and y coordinates refer to the domains of the x and y axes
	fig.add_vrect(x0=bear['start'], x1=bear['end'],
	fillcolor='grey', opacity=0.15, layer="below", line_width=0)
	'''
	fig.add_shape(type="rect",
	x0=bear['start'], x1=bear['end'],
	y0=min(bearData[i]['close']), y1=max(bearData[i]['close']),
	line=dict(
		color="RoyalBlue",
		width=0,
	),
	fillcolor="LightSkyBlue",
	opacity=0.5
	)
	'''

fig.update_yaxes(type='log')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(template = 'none',
	legend_title="Bear Markets<br><i>(starting month)</i>",
	title="Bear markets since The Great Depression<br>S&P500",
	yaxis_title = "S&P500<br>Daily Close (Log Scale)",
	)
fig.show()

In [12]:
currentMarket = df[df['timestamp'] > pd.Timestamp(2022, 1, 2)]
currentMarket['days'] = range(len(currentMarket))
pos = currentMarket.columns.get_loc('close')
currentMarket['change'] =  (currentMarket.iloc[1:, pos] / currentMarket.iat[0, pos]) - 1

C:\Users\ash\AppData\Local\Temp/ipykernel_15944/2968191466.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\ash\AppData\Local\Temp/ipykernel_15944/2968191466.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
currentMarket

,timestamp,volume,open,close,high,low,adjclose,days,change
23614,2022-01-03 14:30:00,2775190000,4778.140137,4796.560059,4796.640137,4758.169922,4796.560059,0,NaN
23615,2022-01-04 14:30:00,3641050000,4804.509766,4793.540039,4818.620117,4774.270020,4793.540039,1,-0.000630
23616,2022-01-05 14:30:00,3733540000,4787.990234,4700.580078,4797.700195,4699.439941,4700.580078,2,-0.020010
23617,2022-01-06 14:30:00,3371250000,4693.390137,4696.049805,4725.009766,4671.259766,4696.049805,3,-0.020955
23618,2022-01-07 14:30:00,3279870000,4697.660156,4677.029785,4707.950195,4662.740234,4677.029785,4,-0.024920
...,...,...,...,...,...,...,...,...,...
23720,2022-06-06 13:30:00,3852050000,4134.720215,4121.430176,4168.779785,4109.180176,4121.430176,106,-0.140753
23721,2022-06-07 13:30:00,3476470000,4096.470215,4160.680176,4164.859863,4080.189941,4160.680176,107,-0.132570
23722,2022-06-08 13:30:00,3090180000,4147.120117,4115.770020,4160.140137,4107.200195,4115.770020,108,-0.141933
23723,2022-06-09 13:30:00,3192330000,4101.649902,4017.820068,4119.100098,4017.169922,4017.820068,109,-0.162354


In [14]:
import plotly.io

plotly.io.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [15]:
bearFigs = go.Figure()

important = ['Dec-1961', 'Mar-2000', 'Oct-2007', 'Feb-2020']

for data in bearData:
    name = data.iloc[0]['timestamp'].strftime("%b-%Y")
    bearFigs.add_trace(go.Scatter(x=data['days'], y=data['change'], mode='lines', name=name, opacity=0.4 if name in important else 0.15 ))

bearFigs.add_trace(go.Scatter(x=currentMarket['days'], y=currentMarket['change'], mode='lines', name='Current Market', line={'width': 3, 'color': 'darkblue'}))

bearFigs.update_layout(xaxis={'range':[0, 1912]}, 
    yaxis=dict(tickformat="0.2%", range=[-0.6, 0.05]),
    legend_title="Bear Markets<br><i>(starting month)</i>",
    title="Where are we compared to previous bear markets?<br>S&P500",
    xaxis_title = "Days since last peak",
    yaxis_title = "Drawdown",
    template='none',
    legend_traceorder="reversed"
)

bearFigs.show()

In [16]:
bearData[0].iloc[0]['timestamp'].strftime("%d-%b-%Y")

'16-Sep-1929'

In [17]:
df

,timestamp,volume,open,close,high,low,adjclose
0,1927-12-30 14:30:00,0,17.660000,17.660000,17.660000,17.660000,17.660000
1,1928-01-03 14:30:00,0,17.760000,17.760000,17.760000,17.760000,17.760000
2,1928-01-04 14:30:00,0,17.719999,17.719999,17.719999,17.719999,17.719999
3,1928-01-05 14:30:00,0,17.549999,17.549999,17.549999,17.549999,17.549999
4,1928-01-06 14:30:00,0,17.660000,17.660000,17.660000,17.660000,17.660000
...,...,...,...,...,...,...,...
23720,2022-06-06 13:30:00,3852050000,4134.720215,4121.430176,4168.779785,4109.180176,4121.430176
23721,2022-06-07 13:30:00,3476470000,4096.470215,4160.680176,4164.859863,4080.189941,4160.680176
23722,2022-06-08 13:30:00,3090180000,4147.120117,4115.770020,4160.140137,4107.200195,4115.770020
23723,2022-06-09 13:30:00,3192330000,4101.649902,4017.820068,4119.100098,4017.169922,4017.820068


In [18]:
monthlyChangeFig = go.Figure()

monthly_change = df.set_index('timestamp').resample('M').last().pct_change()['close']

monthlyChangeFig.add_trace(go.Bar(
    x = df.set_index('timestamp').resample('M').last().index,
    y = monthly_change,
    text = [f"{m:0.2%}" for m in monthly_change],
    textposition = 'outside'
))

monthlyChangeFig.update_layout(yaxis=dict(tickformat=".2%"))

monthlyChangeFig.show()

In [19]:
df.set_index('timestamp').resample('M').last().pct_change()

,volume,open,close,high,low,adjclose
timestamp,,,,,,
1927-12-31,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-31,NaN,-0.005096,-0.005096,-0.005096,-0.005096,-0.005096
1928-02-29,NaN,-0.017644,-0.017644,-0.017644,-0.017644,-0.017644
1928-03-31,NaN,0.117034,0.117034,0.117034,0.117034,0.117034
1928-04-30,NaN,0.024378,0.024378,0.024378,0.024378,0.024378
...,...,...,...,...,...,...
2022-02-28,0.147943,-0.017514,-0.031360,-0.028349,-0.022406,-0.031360
2022-03-31,-0.176745,0.056233,0.035773,0.048812,0.049892,0.035773
2022-04-30,0.042686,-0.075075,-0.087957,-0.072428,-0.089645,-0.087957


In [20]:
currentMarket

,timestamp,volume,open,close,high,low,adjclose,days,change
23614,2022-01-03 14:30:00,2775190000,4778.140137,4796.560059,4796.640137,4758.169922,4796.560059,0,NaN
23615,2022-01-04 14:30:00,3641050000,4804.509766,4793.540039,4818.620117,4774.270020,4793.540039,1,-0.000630
23616,2022-01-05 14:30:00,3733540000,4787.990234,4700.580078,4797.700195,4699.439941,4700.580078,2,-0.020010
23617,2022-01-06 14:30:00,3371250000,4693.390137,4696.049805,4725.009766,4671.259766,4696.049805,3,-0.020955
23618,2022-01-07 14:30:00,3279870000,4697.660156,4677.029785,4707.950195,4662.740234,4677.029785,4,-0.024920
...,...,...,...,...,...,...,...,...,...
23720,2022-06-06 13:30:00,3852050000,4134.720215,4121.430176,4168.779785,4109.180176,4121.430176,106,-0.140753
23721,2022-06-07 13:30:00,3476470000,4096.470215,4160.680176,4164.859863,4080.189941,4160.680176,107,-0.132570
23722,2022-06-08 13:30:00,3090180000,4147.120117,4115.770020,4160.140137,4107.200195,4115.770020,108,-0.141933
23723,2022-06-09 13:30:00,3192330000,4101.649902,4017.820068,4119.100098,4017.169922,4017.820068,109,-0.162354


In [21]:
(currentMarket.iloc[-1]['close'] / currentMarket.iloc[0]['close']) -1

-0.18673798310251433

In [22]:
currentMarket

,timestamp,volume,open,close,high,low,adjclose,days,change
23614,2022-01-03 14:30:00,2775190000,4778.140137,4796.560059,4796.640137,4758.169922,4796.560059,0,NaN
23615,2022-01-04 14:30:00,3641050000,4804.509766,4793.540039,4818.620117,4774.270020,4793.540039,1,-0.000630
23616,2022-01-05 14:30:00,3733540000,4787.990234,4700.580078,4797.700195,4699.439941,4700.580078,2,-0.020010
23617,2022-01-06 14:30:00,3371250000,4693.390137,4696.049805,4725.009766,4671.259766,4696.049805,3,-0.020955
23618,2022-01-07 14:30:00,3279870000,4697.660156,4677.029785,4707.950195,4662.740234,4677.029785,4,-0.024920
...,...,...,...,...,...,...,...,...,...
23720,2022-06-06 13:30:00,3852050000,4134.720215,4121.430176,4168.779785,4109.180176,4121.430176,106,-0.140753
23721,2022-06-07 13:30:00,3476470000,4096.470215,4160.680176,4164.859863,4080.189941,4160.680176,107,-0.132570
23722,2022-06-08 13:30:00,3090180000,4147.120117,4115.770020,4160.140137,4107.200195,4115.770020,108,-0.141933
23723,2022-06-09 13:30:00,3192330000,4101.649902,4017.820068,4119.100098,4017.169922,4017.820068,109,-0.162354


In [138]:
i = -2

fig = go.Figure()
name = bearData[i].iloc[0]['timestamp'].strftime("%b-%Y")
fig.add_trace(go.Scatter(x=currentMarket['days'], y=currentMarket['change'], mode='lines', name='Current Market', line={'width': 3, 'color': 'darkblue'}, 
	hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in currentMarket.iterrows()]
	))
fig.add_trace(go.Scatter(x=bearData[i]['days'], y=bearData[i]['change'], mode='lines', name=name,
	hovertemplate=[f" {x['change']:.1%} ({x['timestamp'].strftime('%b %d, %Y')})" for i, x in bearData[i].iterrows()]
	))

minCandle = bearData[i][bearData[i].close == bearData[i].close.min()]

# Peak
fig.add_annotation(x=0, 
	y=0,
	text=f'''┌ Peak''',
	align='left',
	showarrow=False,
	yshift=9,
	xshift=-5,
	xanchor='left'
	)

# Market bottom
fig.add_annotation(x=minCandle['days'].values[0], 
	y=minCandle['change'].values[0],
	text=f'''└ {minCandle['change'].values[0]:.1%}, {(minCandle.iloc[0]['timestamp']- bearData[i].iloc[0]['timestamp']).days} days since peak ({minCandle.iloc[0]['days']} trading days)''',
	align='left',
	showarrow=False,
	yshift=-8,
	xshift=-6,
	xanchor='left'
	)

# Cycle end
fig.add_annotation(x=bearData[i].iloc[-1]['days'], 
	y=bearData[i].iloc[-1]['change'],
	text=f'''{(bearData[i].iloc[-1]['timestamp']- bearData[i].iloc[0]['timestamp']).days} days since peak ({bearData[i].iloc[-1]['days']} trading days)   <br>
	{(bearData[i].iloc[-1]['timestamp']- minCandle.iloc[0]['timestamp']).days} days since market bottom ({bearData[i].iloc[-1]['days'] - minCandle.iloc[0]['days']} trading days) ┐''',
	align='right',
	showarrow=False,
	yshift=17,
	xshift=7,
	xanchor='right',
	ax=0, ay=0
	)

fig.update_layout( 
	yaxis=dict(tickformat="0.2%"),
	xaxis_title = "Trading days since last peak 🠆",
	yaxis_title = "🠄 Drawdown",
	template='simple_white',
	paper_bgcolor="rgb(255,255,255)", plot_bgcolor="rgb(255,255,255)",
	legend_traceorder="reversed",
	hovermode='x unified',
	margin=dict(t=50),
	width=800
)
fig.show()

In [26]:
i = -1
bearData[i][bearData[i].close == bearData[i].close.min()]

,timestamp,volume,open,close,high,low,adjclose,days,change
23164,2020-03-23 13:30:00,7402180000,2290.709961,2237.399902,2300.72998,2191.860107,2237.399902,23,-0.33925


In [119]:
[x for i, x in currentMarket.iterrows()]

[timestamp    2022-01-03 14:30:00
 volume                2775190000
 open                 4778.140137
 close                4796.560059
 high                 4796.640137
 low                  4758.169922
 adjclose             4796.560059
 days                           0
 change                       NaN
 Name: 23614, dtype: object,
 timestamp    2022-01-04 14:30:00
 volume                3641050000
 open                 4804.509766
 close                4793.540039
 high                 4818.620117
 low                   4774.27002
 adjclose             4793.540039
 days                           1
 change                  -0.00063
 Name: 23615, dtype: object,
 timestamp    2022-01-05 14:30:00
 volume                3733540000
 open                 4787.990234
 close                4700.580078
 high                 4797.700195
 low                  4699.439941
 adjclose             4700.580078
 days                           2
 change                  -0.02001
 Name: 23616, dtype: obj